## Live Project 

### Objective 
Sales director of the company is tasked with developing key sales statistical and analytical insights to grow the company to make informed business decisions. These analyses serves as evaluating agent performance, revenue, expenses and probability. These insights will help them to steer the agency towards sustained growth and success in dynamic entertainment landscape.


## Establishing SQL connection to Machine Learning

In [36]:
import pandas as pd
from sqlalchemy import create_engine # Takes in the connection URL and returns SQLALchemy Engine
import urllib #Handling URLs, parsing them and sending HTTP request

# DB credentials
server = 'qaececrm-s1.database.windows.net'  
database = 'QAECECRM_June2025'
username = 'dbadmin'
password = 'DashTech1234'
driver = 'ODBC Driver 17 for SQL Server'

# Create connection string
params = urllib.parse.quote_plus(
    f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

### Collecting and Generating the Tables From the Database

In [37]:
from sqlalchemy import text #to run raw SQL queries

def get_all_table(engine):
    with engine.connect() as conn:
        result = conn.execute(text("""SELECT t.name as table_name
                                  FROM sys.tables t
                                  ORDER BY t.name"""))
        table_list = [row[0] for row in result]
        table_df = pd.DataFrame(table_list, columns=['Name of the tables'])
    return table_df    


table_df = get_all_table(engine)
#table_list = table_df['Name of the tables'].tolist()
#print(table_list)
#Count of the tables
print(f"Total number of tables presented: {len(table_df)}")

table_df

Total number of tables presented: 189


,Name of the tables
0,AgentCommission
1,AgentCommissionProgram
2,AgentCommissionTypeData
3,AgentPayrollLog
4,Alert
...,...
184,VenueDescription
185,VenueGenreType
186,VenueRequirement
187,VenueRequirementNotification


**Insights**

In [38]:
from sqlalchemy import text

def run_query(engine, query):
    with engine.connect() as conn:
        df = pd.read_sql(text(query), conn)
    return df

### 1) Offer Volume Breakdown by Year and Month

In [4]:
q1 = """
SELECT 
    YEAR(CreatedDate) AS Year,
    MONTH(CreatedDate) AS Month,
    COUNT(*) AS OfferCount
FROM Offer
GROUP BY YEAR(CreatedDate), MONTH(CreatedDate)
ORDER BY Year, Month;
"""
df_q1 = run_query(engine, q1)
df_q1.head()

,Year,Month,OfferCount
0,2018,6,16
1,2018,7,9
2,2018,8,2
3,2018,12,2
4,2020,5,2


### 2) Top Presenters by Ticketed Events

Find which presenters are hosting the most revenue-potential events.

In [5]:
q2 = """SELECT 
    PresenterName, 
    COUNT(*) AS TicketedEventCount
FROM Offer
WHERE IsTicketedEvent = 1
GROUP BY PresenterName
ORDER BY TicketedEventCount DESC
"""
df_q2 = run_query(engine, q2)
df_q2.head()

,PresenterName,TicketedEventCount
0,ROANOKE RAPIDS THEATRE,6
1,"Hillbilly Christmas in July, Inc.",1
2,HOWARD PERFORMING ARTS CENTER,1
3,Kris Kaminski - Test Offer,1
4,MAJESTIC THEATRE,1


### 3) Offer Creation Trend by Day of Week

In [33]:
q3 = """SELECT 
    DATENAME(WEEKDAY, CreatedDate) AS DayOfWeek,
    COUNT(*) AS OfferCount
FROM Offer
GROUP BY DATENAME(WEEKDAY, CreatedDate)
ORDER BY OfferCount DESC;
"""

df_q3 = run_query(engine, q3)
df_q3.head()

,DayOfWeek,OfferCount
0,Monday,13
1,Saturday,9
2,Friday,8
3,Tuesday,6
4,Thursday,2


### 4)Top Cities by Average Venue Capacity

This can guide event planning or city-level business focus.

In [7]:
q4 = """SELECT 
    MailingCity, 
    AVG(VenueCapacity) AS AvgCapacity,
    COUNT(*) AS VenueCount
FROM Offer
WHERE VenueCapacity IS NOT NULL
GROUP BY MailingCity
ORDER BY AvgCapacity DESC
"""
df_q4 = run_query(engine, q4)
df_q4.head()

,MailingCity,AvgCapacity,VenueCount
0,MOUNTAIN HOME,1650,2
1,ROANOKE RAPIDS,1500,8
2,Prestonsburg,1100,2
3,Lexington,1000,1
4,Montevallo,800,1


## TOP Venues by Number Of Events

In [57]:
q5 = """
SELECT VenueName, COUNT(BlueCardId) AS EventCount
FROM BlueCard
WHERE VenueName IS NOT NULL AND VenueName <> ''
GROUP BY VenueName
ORDER BY EventCount DESC;
"""

df_q5 = run_query(engine, q5)
df_q5.head()

,VenueName,EventCount
0,TBD,1673
1,Private Residence,768
2,The Breakers,357
3,THE FARM AT OLD EDWARDS,234
4,Private Home,221
